[![Video Placeholder](./placeholder.png)](https://youtu.be/6d0-8BbBvoM "Title Test")


# Making native commands more like PowerShell cmdlets

PowerShell cmdlets are light-weight commands that can be assembled together to perform powerful
management operations. To reduce the overhead of learning a brand new command each time a new one
ships, cmdlets follow a very specific design pattern to reduce the amount of new information that a
user must learn before using the cmdlet.

In recent years, the rapid introduction of additional complex native commands, such as Kubectl, offer extensive functionality but lack the ease of use that PowerShell users have come to expect. There are reimplementation methods that module developers can pursue to solve this, but often the benefit is outweighed by the cost of development and the inevitable updates when the original native command is updated.

This is an inside look at work we are thinking about to make native commands more PowerShell-like while reducing the cost of ownership. We value your feedback during this early stage of development.

## What you need to get started

1. If you haven't seen Jim Truher's blog detailing this work and wish to know more:
[Native Command Blog](http://example.com "Title")

2. If you want to try this demonstration, you will need Kubernetes. If you already have a
   safe-for-experiment implementation, you are all set. We are using Kubernetes on Docker and it can
   be downloaded from here:[Kubernetes on Docker](https://www.docker.com/products/kubernetes)

3. If you download Docker, you will need to enable Kubernetes on Docker. We have those steps, and
   optional Fast-safe configuration instructions here: [Additional Configuration](https://)

4. Download the latest version of the module Microsoft.PowerShell.KubeCtl from:
   [Get the module](https://) and place in your module folder.

5. To work with the Kubectl cmdlets, import the Microsoft.PowerShell.KubeCtl Module.

```powershell
Import-Module Microsoft.PowerShell.KubeCtl
```

To import the module automatically when PowerShell starts, add the Import-Module statement to your
PowerShell Profile.

In [1]:
Import-Module Microsoft.PowerShell.KubeCtl #Try It!

**Note** - you must run the above Import-Module command when you open this notebook to load the KubeCtl cmdlets. Without this module loaded, example commands below will not work.

## Where to leave comments

We appreciate and need your feedback. Our primary question is - is this framework for enabling native commands the right direction? Are there other ideas we should be exploring?  Please leave us feedback here: [Feedback Link](http://)

## Why are we talking about this?

 Native commands, for the most part, just work in PowerShell. Many of those commands are complex and take time to learn. If you want to have the familiar, comfortable PowerShell experience, there are only a few options that are available to us.

 - You can re-implement the tool in managed code or script
 - You can call web based apis. SWAGGER provides a very easy way to do this
   - <https://github.com/Azure/autorest> is one example
 - You can wrap the native application in a powershell script
 - I can just use the tool as is without change

 We are exploring a **new** way of handling native commands to make them more like PowerShell cmdlets without some of the drawbacks.  Note - the benefits and drawbacks, plus the details of our proposal is discussed shortly in Part 2. 
 
 First, is it worth making native commands more PowerShell like?  

## Why would you want native commands more like PowerShell?

Kubectl is a rich native command that offers extensive help. Once you understand the pattern of usage, finding documentation and examples is fairly simple. Run the **KubeCtl** command to start exploring.

In [ ]:
kubectl 

Experienced PowerShell users benefit from KubeCtl being more like PowerShell cmdelets. As an example, To list all the available cmdlets for the KubeCtl module, a PowerShell user may try: **Get-Command -Module Microsoft.PowerShell.KubeCtl**

In [ ]:
Get-Command -Module Microsoft.PowerShell.KubeCtl

Sinilarly, A PowerShell user attemping to discover cmdlets for retreiving information from Kubernetes might try:

```powershell
Get-Kube<tab>
```

**Note** - The tab key triggers PowerShell's TabExpansion.  This will provide all the Get cmdlets and thier respective nouns.

### PowerShell users expect to use what they know - PowerShell

KubeCtl can display a list of pods

```powershell
kubectl get pods
```


In [ ]:
kubectl get pods

Note - nothing is listed in my example because I haven't created any pods - but there are pods that exist, you just don;t know about them unless you know how to use the commmmand.

```powershell
kubectl get pods --all-namespaces
```

In [ ]:
kubectl get pods --all-namespaces

PoewrShell users expect Get cmdlets to return all results unless filtered. A PowerShell user might try **Get-KubePod**


In [ ]:
Get-KubePod

KubeCtl can list pods by namespace: **kubectl get pods -n kube-system**

In [ ]:
kubectl get pods -n kube-system

PowerShell users should be able to filter results using the common PowerSehll practices they already know. In this case, If I wanted only the kube-system pods, I could filter the results using a **Where-Object**.

In [ ]:
get-kubepod | where namespace -eq 'kube-system'

PoweShell continues to help me get the information I need, even if I don;t fully understand the native the command or the underlying technology. Here, I was trying to get a list of pods sorted by restart counts. After seraching the web, I started to try this:

```powershell
kubectl get pods --all-namespaces --sort-by=.status.containerStatuses[0].restartcount
```
I didn't understand the command and wasn't getting the results I desired, so I switch to the trusted PowerShell knowledge I already know.
```powershell
Get-KubePod | Get-Member
# This lists the properties of the Pod object - yup - everything is an object!
# I noticed the object Restarts - the rest was easy
Get-KubePod | Sort-Object -Property Restarts
# I wanted the restarts to be displayed at the top of the list, so I changed the sort
Get-KubePod | Sort-Object -Property Restarts -Descending
```



In [ ]:
Get-KubePod | Sort-Object -Property Restarts -Descending

**The point - Native commands, like KubeCtl, are gaining in popularity and complexity making it more challenging to learn and adopt important technologies. By bringing these native commands into the PowerShell Ecosystem, PowerShell users can take the knowledge they already have and imediatly start to be successfull and moving the business forward.** 

## PART 2

_________________

## Problems of functionality coverage when using PowerShell

### Solution Options

 To achieve coverage for tools that participate fully in the PowerShell ecosystem, only a few options exist:

 - You can re-implement the tool in managed code or script
 - You can call web based apis. SWAGGER provides a very easy way to do this
   - <https://github.com/Azure/autorest> is one example
 - You can wrap the native application in a powershell script
 

## Reimplementation

 There a many benefits in a complete rewrite of a command:

 - The expression of behavior can be made more "native" to the new environment
 - Performance issues can be addressed
 - New code means that new technologies can be used advantageously

 ### Issues with Reimplementations

 The biggest issue with reimplementation is probably the amount of work that is needed to achieve behavior expressed in the original.
 This is especially the case if the reimplementor is not intimately familiar with the workings of the tool.

 Another issue with reimplementation is that you need to continue to track changes in the original code.
 This can be a challenge as depending on the activity and updates in the tool, wholescale changes can occur that then need to be reimplemented,
 or the reimplementation will be out of date.
 Worse, if the the command is the client side of a client/server app, changes in the server may negatively effect the reimplementation.

 


 ## API wrapping

 Many native apps use a REST endpoint to retrieve data.
 These can be used to interact with the data end point, retrieve data from it and then present it to the user.
 
 ### Issues with API wrapping
 The most impactful issues with this approach are about authentication and complexity.
 Also, simple API wrapping generally results in a command that is _developer_ rather than _administrator_ focused.
 There is quite a bit of logic wrapped up in a command to avoid just calling the API.

## Native Application Wrapping

 Because it is possible to call native applications easily from within PowerShell it is possible to write a script that provides a more PowerShell-like experience.
 It can provide parameter handling such as prompting for mandatory parameters and tab-completion for parameter values.
 It can take the application output and use the text output into objects so it can take advantage of all the post processing tools such as `Sort-Object`, `Where-Object`, etc.

 
 ### Issues with native application wrapping

 The issues are roughly the same as above, there is a certain amount of programming that is needed to call the application.
 There is some programming needed to convert the text output to objects so they can participate in the PowerShell pipelines.
 
 

__________

## Is there a better way?

 It may be possible to create a framework that inspects the help of the application and _automatically_ create the code that calls the underlying application.
 This framework can also handle the output mapping to an object more suitable for the PowerShell environment.

## First Experiment - Microsoft.PowerShell.Kubectl Module

There were some problems that I wanted to solve with this first experiment

 - wrap `kubectl api-resources` in a function
   - automatically create object output from `kubectl api-resources`
 - Auto-generate functions for each resource that could be retrieved (only resource get for now)
   - only support `name` as a parameter
 - Auto-generate the conversion of output to objects to look similar to the usual `kubectl` output

## Second Experiment - Module KubectlHelpParser

 I wanted to see if I could read any help content from `kubectl` that would enable me to auto-generate a complete
 proxy to the `kubectl` command that included general parameters, command specific parameters, and help.
 It turns out that `kubectl` help is regular enough where this is quite possible.


## Call To Action

 First, I'm really interested if having a framework that can autogenerate functions that wrap a native executable is useful? The obvious response might be "of course", but how much of a solution is really needed to provide value?

 Second, I would _really_ like to know if you would like us to investigate _specific_ tools for this sort of treatment.
 If it is possible to make this a generic framework, I would love to have more examples of tools which would be beneficial
 to you and test our ability to handle.

 ## Please provide feedback!

 Here is the [link for feedback](http://TODO)